In [1]:
import tensorflow as tf
import os
from tensorflow.keras import layers, regularizers

In [2]:
dataset_path ="dataset/data"
bad_images =0
data_classes = ["Blight","Common_Rust","Gray_Leaf_Spot","Healthy"]
for classname in data_classes:
    folder_path = os.path.join("..",dataset_path, classname)
    for fname in os.listdir(folder_path):
        file = os.path.join(folder_path,fname)
        try:
            img_bytes = tf.io.read_file(file)
            tf.io.decode_image(img_bytes)
        except Exception as e:
            print(f"Failed to decode an image at {file} deleting the image")
            bad_images +=1
            os.remove(file)
print(f"total images deleted are {bad_images}")
    

total images deleted are 0


In [3]:
image_size =(256,256)
batch_size =64

In [4]:
training_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join("..", dataset_path),
    subset ="training",
    validation_split =0.2,
    seed= 123,
    image_size = image_size,
    batch_size = batch_size,
    label_mode ="int"
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join("..", dataset_path),
    subset ="validation",
    validation_split =0.2,
    seed= 123,
    image_size = image_size,
    batch_size = batch_size,
    label_mode ="int"
)

Found 4188 files belonging to 4 classes.
Using 3351 files for training.
Found 4188 files belonging to 4 classes.
Using 837 files for validation.


In [5]:
data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical", name="aug_flip"),
    tf.keras.layers.RandomRotation(factor=0.2, name="aug_rotation"),
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2, name="aug_zoom"),
    tf.keras.layers.RandomBrightness(factor=0.2, name="aug_brightness"),
    tf.keras.layers.RandomContrast(factor=0.2, name="aug_contrast"),                 
], name="data_augmentation")

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(*image_size,3)),
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_1_1"),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_1_1_1"),
    layers.BatchNormalization(name="bn_1_1"),
    layers.Activation("relu"),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_1_2"),
    layers.BatchNormalization(name="bn_1_2"),
    layers.Activation("relu"),
    layers.MaxPooling2D(pool_size=(2,2), name="pool_1"),
    layers.Dropout(0.25, name="dropout_conv_1"),

    
    layers.Conv2D(filters=128, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_2_1"),
    layers.BatchNormalization(name="bn_2_1"),
    layers.Activation("relu"),
    layers.Conv2D(filters=128, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_2_2"),
    layers.BatchNormalization(name="bn_2_2"),
    layers.Activation("relu"),
    layers.MaxPooling2D(pool_size=(2,2), name="pool_2"), 
    layers.Dropout(0.25, name="dropout_conv_2"),

    
    layers.Conv2D(filters=256, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_3_1"),
    layers.BatchNormalization(name="bn_3_1"),
    layers.Activation("relu"),
    layers.Conv2D(filters=256, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2(),
                  name="conv_3_2"),
    layers.BatchNormalization(name="bn_3_2"),
    layers.Activation("relu"),
    layers.MaxPooling2D(pool_size=(2,2), name="pool_3"), 
    layers.Dropout(0.3, name="dropout_conv_3"),

    layers.GlobalAveragePooling2D(name="global_avg_pool"),

    layers.Dense(units=256, activation="relu",
                 kernel_regularizer=regularizers.l2(),
                 name="dense_1"),
    layers.Dropout(0.4, name="dropout_dense_1"),

    layers.Dense(units=128, activation="relu",
                 kernel_regularizer=regularizers.l2(),
                 name="dense_2"),
    layers.Dropout(0.3, name="dropout_dense_2"),

    layers.Dense(units=4, activation="softmax", name="output_layer")
])
model.compile(
    optimizer ="adam", metrics =["accuracy"],loss="sparse_categorical_crossentropy"
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_1 (Conv2D)               │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_1_1 (Conv2D)             │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1_1 (BatchNormalization)     │ (None, 256, 256, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_2 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1_2 (BatchNormalization)     │ (None, 256, 256, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv_1 (Dropout)        │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_1 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2_1 (BatchNormalization)     │ (None, 128, 128, 128)  │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_2 (Conv2D)               │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2_2 (BatchNormalization)     │ (None, 128, 128, 128)  │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv_2 (Dropout)        │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_1 (Conv2D)               │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3_1 (BatchNormalization)     │ (None, 64, 64, 256)    │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_2 (Conv2D)               │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3_2 (BatchNormalization)     │ (None, 64, 64, 256)    │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 64, 64, 256)    │             

 Total params: 1,285,124 (4.90 MB)

 Trainable params: 1,283,332 (4.90 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [7]:
save_path= os.path.join("..", "training_models/best_model_version.keras")
log_dr = os.path.join("..", "logs")

In [ ]:
model.fit(
    training_dataset,
    epochs =20,
    verbose=1,
    validation_data = validation_dataset,
    callbacks =[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=5,
            mode="max",
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            save_path,
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1,
            mode="max"
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dr,
            histogram_freq=1,
            write_graph=True,
            write_images=False
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2,         
            patience=5,         
            min_lr=1e-7,        
            verbose=1,
            mode='min'
        )]
)

Epoch 1/20
 2/53 ━━━━━━━━━━━━━━━━━━━━ 4:34:35 323s/step - accuracy: 0.2578 - loss: 13.9929